# Inroduction


This sample notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

Please note that the pricing model used in this notebook is a simple baseline. You are expected to design and implement a more advanced and effective model.


In [7]:
!pip install pathway bokeh panel --quiet

print("Hello")

Hello


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
import numpy as np
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn
from bokeh.io import output_notebook, show

print("Hello")

Hello


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 1: Importing and Preprocessing the Data

In [ ]:
# Load the dataset (replace with your actual file if needed)
df = pd.read_csv('dataset.csv')

# Combine date and time columns if needed
if 'LastUpdatedDate' in df.columns and 'LastUpdatedTime' in df.columns:
    df['Timestamp'] = pd.to_datetime(
        df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
        format='%d-%m-%Y %H:%M:%S'
    )
else:
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Sort by time
df = df.sort_values('Timestamp').reset_index(drop=True)

# Ensure a unique lot identifier column (LotID)
if 'LotID' not in df.columns:
    df['LotID'] = df['Latitude'].astype(str) + '_' + df['Longitude'].astype(str)

# Save a streaming-ready CSV for Pathway
df[["Timestamp", "Occupancy", "Capacity", "LotID"]].to_csv("parking_stream.csv", index=False)

print("Hello")
df.columns

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

In [ ]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    LotID: str
print("Hello World")

In [ ]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=100
)
print("Hello")

In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


print("Hello")

# Step 2: Making a simple pricing function

In [ ]:
# --- Model 2: Demand-Based Price Function ---

# 1. Ensure all relevant columns are numeric
cols_to_numeric = ['Occupancy', 'Capacity', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay']
for col in cols_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 2. Assign weights for vehicle types
vehicle_type_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_type_weights).fillna(1.0)

# 3. Normalize features (optional, for stability)
df['Norm_Occupancy'] = df['Occupancy'] / df['Capacity']
df['Norm_Queue'] = df['QueueLength'] / df['QueueLength'].max()
df['Norm_Traffic'] = df['TrafficConditionNearby'] / df['TrafficConditionNearby'].max()

# 4. Set coefficients (tune as needed)
ALPHA = 1.0
BETA = 0.5
GAMMA = 0.3
DELTA = 0.8
EPSILON = 0.7

# 5. Compute raw demand
df['Raw_Demand'] = (
    ALPHA * df['Norm_Occupancy'] +
    BETA * df['Norm_Queue'] -
    GAMMA * df['Norm_Traffic'] +
    DELTA * df['IsSpecialDay'] +
    EPSILON * df['VehicleTypeWeight']
)

# 6. Normalize demand to [0, 1]
if df['Raw_Demand'].max() != df['Raw_Demand'].min():
    df['Norm_Demand'] = (df['Raw_Demand'] - df['Raw_Demand'].min()) / (df['Raw_Demand'].max() - df['Raw_Demand'].min())
else:
    df['Norm_Demand'] = 0.0

# 7. Compute price
BASE_PRICE = 10.0
LAMBDA = 1.0

df['Price_Model2'] = BASE_PRICE * (1 + LAMBDA * df['Norm_Demand'])
df['Price_Model2'] = df['Price_Model2'].clip(lower=BASE_PRICE*0.5, upper=BASE_PRICE*2)

# 8. Save for streaming/visualization
df[["Timestamp", "Occupancy", "Capacity", "LotID", "Price_Model2"]].to_csv("parking_stream_model2.csv", index=False)

print("Model 2 demand-based pricing calculated and saved.")




# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [ ]:
# # # Activate the Panel extension to enable interactive visualizations
# # pn.extension()

# # Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
# def price_plotter(source):
#     # Create a Bokeh figure with datetime x-axis
#     fig = bokeh.plotting.figure(
#         height=400,
#         width=800,
#         title="Pathway: Daily Parking Price",
#         x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
#     )
#     # Plot a line graph showing how the price evolves over time
#     fig.line("t", "price", source=source, line_width=2, color="navy")

#     # Overlay red circles at each data point for better visibility
#     fig.circle("t", "price", source=source, size=6, color="red")

#     return fig

# # Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# # - 'price_plotter' is the rendering function
# # - 'sorting_col="t"' ensures the data is plotted in time order
# viz = delta_window.plot(price_plotter, sorting_col="t")

# # Create a Panel layout and make it servable as a web app
# # This line enables the interactive plot to be displayed when the app is served
# pn.Column(viz).servable()
# print("Hello")



# import panel as pn
# import bokeh.plotting

# pn.extension('bokeh')

# def price_plotter(source):
#     fig = bokeh.plotting.figure(
#         height=400,
#         width=800,
#         title="Model 2: Demand-Based Price per Lot (Real-Time)",
#         x_axis_type="datetime",
#     )
#     lots = list(set(source.data['LotID']))
#     colors = bokeh.palettes.Category10[10] + bokeh.palettes.Category20[20]
#     legend_added = False  # Track if at least one legend is added
#     for i, lot in enumerate(lots):
#         mask = [l == lot for l in source.data['LotID']]
#         # Only plot if this lot has data
#         if any(mask):
#             fig.line(
#                 [t for t, m in zip(source.data['t'], mask) if m],
#                 [p for p, m in zip(source.data['Price_Model2'], mask) if m],
#                 line_width=2,
#                 color=colors[i % len(colors)],
#                 legend_label=f"Lot {lot}"
#             )
#             legend_added = True
#     if legend_added:
#         fig.legend.location = "top_left"
#     fig.xaxis.axis_label = 'Time'
#     fig.yaxis.axis_label = 'Price ($)'
#     return fig

# viz = data_with_time.plot(price_plotter, sorting_col="t")
# pn.Column(viz).servable()



from bokeh.plotting import figure, show, output_notebook
import bokeh.palettes

output_notebook()

def plot_model2(df):
    fig = figure(
        height=400,
        width=800,
        title="Model 2: Demand-Based Price per Lot",
        x_axis_type="datetime",
    )
    lots = df['LotID'].unique()
    palette = bokeh.palettes.Category10[10] + bokeh.palettes.Category20[20]
    for i, lot in enumerate(lots):
        lot_df = df[df['LotID'] == lot]
        fig.line(
            lot_df['Timestamp'],
            lot_df['Price_Model2'],
            line_width=2,
            color=palette[i % len(palette)],
            legend_label=f"Lot {lot}"
        )
    fig.legend.location = "top_left"
    fig.xaxis.axis_label = 'Time'
    fig.yaxis.axis_label = 'Price ($)'
    show(fig)

plot_model2(df)




In [ ]:


# # Start the Pathway pipeline execution in the background
# # - This triggers the real-time data stream processing defined above
# # - %%capture --no-display suppresses output in the notebook interface

# %%capture --no-display
pw.run()
print("Hello")

